In [76]:
import glob as glob 
import re as re
from openai import OpenAI

In [37]:
def extract_text_from_srt(srt_lines):
    text_lines = []
    for line in srt_lines:
        # Ignore lines that are timestamps or numbers
        if not re.match(r'^\d+$', line.strip()) and not re.match(r'^\d{2}:\d{2}:\d{2}', line.strip()):
            text_lines.append(line.strip())
    return text_lines

def extract_values(file_path):
    n_value = re.search(r'n-(\d+)', file_path)
    pxpy_value = re.search(r'(p\d+p\d+)', file_path)
    px_value = re.search(r'u-p(\d+)', file_path)
    
    n_value = n_value.group(1) if n_value else None
    pxpy_value = pxpy_value.group(1) if pxpy_value else None
    px_value = px_value.group(1) if px_value else None
    
    return n_value, pxpy_value, px_value


In [41]:
path = "C:/Users/dazaz/Documents/ducksoup/Development/repos/ducksoup_analysis/mkUltimatum_Game_Conflict1_subtitles/"
pattern = "*.srt"
filepaths = glob.glob(path + pattern)
filepaths = [file_path for file_path in filepaths if "audio-dry" not in file_path]

round = r'n-(\d+)'
dyad = r'(p\d+p\d+)'
player = r'p(\d+)'

transcript_dict = {
}
unique_dyads = []

for file_path in filepaths:

    with open(file_path, 'r', encoding='utf-8') as file:
        srt_content = file.readlines()
    # Extract and format the text
    srt_text_lines = extract_text_from_srt(srt_content)
    formatted_text = ' '.join(srt_text_lines)
    formatted_text = re.sub(r'\s+', ' ', formatted_text)  # Remove extra whitespace
    round, dyad, player = extract_values(file_path)
    if dyad not in unique_dyads:
        unique_dyads.append(dyad)
     # Initialize intermediate dictionaries if they don't exist
    if round not in transcript_dict:
        transcript_dict[round] = {}
    if dyad not in transcript_dict[round]:
        transcript_dict[round][dyad] = {}
    if player not in transcript_dict[round][dyad]:
        transcript_dict[round][dyad][player] = {}

    transcript_dict[round][dyad][player] = formatted_text

print(transcript_dict)

{'7': {'p3p7': {'7': "0:00:01.100000-->0:00:16.200000 Hello. I like, I have a pen for you and a pen for me. Come on, push that up a little 0:00:16.200000-->0:00:21.720000 bit. Oh, I don't know. ", '3': "0:00:00-->0:00:24.880000 Hi, hello, hi. I'm going to give you seven. Seven is fair. I think seven is fair. "}, 'p1p10': {'10': '0:00:01.250000-->0:00:06.090000 Hello. What are you thinking of otherwise? 0:00:06.090000-->0:00:15.420000 Ooh, three coins. Would you be willing to at least push that to five coins, at least? 0:00:15.420000-->0:00:19.270000 Five coins? Awesome then, thank you, I appreciate that. ', '1': "0:00:00-->0:00:18.420000 I'm thinking three coins. Yeah. Sure. Let's go for five. Yeah "}, 'p4p8': {'4': '0:00:01.100000-->0:00:10.040000 Hi. Would you take five coins? Would you take five coins? 0:00:16.420000-->0:00:25.690000 Objective goes to less than ten. OK. Fair enough. Fair enough. ', '8': "0:00:02.350000-->0:00:02.710000 Hiya. 0:00:04.590000-->0:00:05.090000 Just chec

In [48]:
# Regex pattern to match and capture the two numeric values
pattern = re.compile(r'p(\d+)p(\d+)')
# Function to extract values
def extract_numeric_values(dyad):
    pattern = re.compile(r'p(\d+)p(\d+)')
    match = pattern.match(dyad)
    if match:
        # Extract the numbers and convert them to integers
        num1, num2 = map(int, match.groups())
        # Sort the numbers so that the smaller number is always first
        return tuple(sorted([num1, num2]))
    return None

for dyad in unique_dyads:
    print(extract_numeric_values(dyad))

{'7': {'p3p7': {'7': "0:00:01.100000-->0:00:16.200000 Hello. I like, I have a pen for you and a pen for me. Come on, push that up a little 0:00:16.200000-->0:00:21.720000 bit. Oh, I don't know. ", '3': "0:00:00-->0:00:24.880000 Hi, hello, hi. I'm going to give you seven. Seven is fair. I think seven is fair. "}, 'p1p10': {'10': '0:00:01.250000-->0:00:06.090000 Hello. What are you thinking of otherwise? 0:00:06.090000-->0:00:15.420000 Ooh, three coins. Would you be willing to at least push that to five coins, at least? 0:00:15.420000-->0:00:19.270000 Five coins? Awesome then, thank you, I appreciate that. ', '1': "0:00:00-->0:00:18.420000 I'm thinking three coins. Yeah. Sure. Let's go for five. Yeah "}, 'p4p8': {'4': '0:00:01.100000-->0:00:10.040000 Hi. Would you take five coins? Would you take five coins? 0:00:16.420000-->0:00:25.690000 Objective goes to less than ten. OK. Fair enough. Fair enough. ', '8': "0:00:02.350000-->0:00:02.710000 Hiya. 0:00:04.590000-->0:00:05.090000 Just chec

In [88]:
client = OpenAI(
    api_key = "sk-None-2zzxakQkR57oRj1SIopxT3BlbkFJoC7pgM3MKYeltGYF1Qbm"
)

for round in range(1,2): 
    dyads = transcript_dict[f"{round}"].keys()
    for dyad in dyads:
        numeric_dyad = extract_numeric_values(dyad)
        proposer     = numeric_dyad[0]
        responder    = numeric_dyad[1]
        
        proposer_transcript  = transcript_dict[f"{round}"][dyad][f"{proposer}"]
        responder_transcript = transcript_dict[f"{round}"][dyad][f"{responder}"]

        prompt = f"""
        Your goal is to extract offers and counter offers from the transcript below. 
        The transcipt is from a dyadic negotiation between a proposer and a responder in the Ultimatum game.
        The structure of the output should be:

        First offer : numeric value.
        Counter offer : numeric value.
        and so on....

        You find the transcript between two players below:
        {proposer_transcript}
        {responder_transcript}
        """

        chat_completion = client.chat.completions.create(
            messages = [
                {
                    "role": "user",
                    "content": prompt 
                }
            ],
            model = "gpt-3.5-turbo"
        )

        print(f"{round}", chat_completion.choices[0].message.content)
    

1 First offer (player x): 15 points.
Counter offer (player y): 10 points each.
1 First offer (Player 1): 20 coins.
Counter offer (Player 2): 3 coins.
1 First offer (player x): 4 coins.
Counter offer (player y): 8 coins.
Counter offer (player x): 10 coins.
1 First offer (Player 1): 3 coins.
Counter offer (Player 2): Split the coins.
1 First offer (Player 1): 5 coins.
Counter offer (Player 2): 9 coins.
Counter offer (Player 1): 9 coins.
Counter offer (Player 2): 10 coins.
